In [1]:
cd ..

/Users/dreamgonfly/Documents/pozalabs/infinityWord


In [2]:
from datasets.seq2seq_datasets import Seq2SeqIndexedDataset

In [3]:
from models.encoders.rnn_encoder import GRUEncoder
from models.decoders.rnn_decoder import GRUDecoder
from models.rnn_model import RNNModel

In [4]:
from trainers.seq2seq_trainer import Seq2SeqTrainer, seq2seq_collate_fn

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

In [6]:
from utils import get_logger

In [7]:
import numpy as np

In [8]:
from datasets.base_datasets import SentencesTokenizedDataset

In [9]:
from dictionaries import BaseDictionary

In [ ]:
len(arg='hi')

In [10]:
dictionary = BaseDictionary.load('base_dictionary')
dictionary.vocabulary_size

48034

In [11]:
device = torch.device('cpu')

In [12]:
embeddings = nn.Embedding(num_embeddings=dictionary.vocabulary_size, embedding_dim=128)
encoder = GRUEncoder(embeddings, hidden_size=256)
decoder = GRUDecoder(embeddings, hidden_size=256)
model = RNNModel(encoder, decoder)
logger = get_logger(log_name='gru_encoder_decoder')
trainer = Seq2SeqTrainer(model=model, 
                         train_dataloader=DataLoader(Seq2SeqIndexedDataset('train'), batch_size=32, collate_fn=seq2seq_collate_fn), 
                         val_dataloader=DataLoader(Seq2SeqIndexedDataset('val'), batch_size=32, collate_fn=seq2seq_collate_fn),
                         loss_function=CrossEntropyLoss(reduce=False),
                         optimizer=Adam(model.parameters()),
                         device=device,
                         logger=logger)

In [13]:
trainer.run(epochs=10)

  0%|          | 0/817 [00:00<?, ?it/s]


[INFO] 07-31 16:25:08 > Epoch: 0   Progress: 0.0% (0:05:31.889677) Train Loss: 10.7992 Val Loss: 8.59461 Train Perplexity: 4.9e+04 Val Perplexity: 5.4e+03 Learning rate: 0.001 
[INFO] 07-31 16:25:08 > Saving the model...


100%|██████████| 817/817 [38:42<00:00,  2.84s/it]


[INFO] 07-31 17:09:46 > Epoch: 1   Progress: 10.0% (0:50:09.796649) Train Loss: 6.8636 Val Loss: 6.76534 Train Perplexity: 9.57e+02 Val Perplexity: 8.67e+02 Learning rate: 0.001 


100%|██████████| 817/817 [22:33<00:00,  1.66s/it]


[INFO] 07-31 17:38:10 > Epoch: 2   Progress: 20.0% (1:18:33.655356) Train Loss: 6.19578 Val Loss: 6.79507 Train Perplexity: 4.91e+02 Val Perplexity: 8.93e+02 Learning rate: 0.001 


100%|██████████| 817/817 [26:08<00:00,  1.92s/it]


[INFO] 07-31 18:09:53 > Epoch: 3   Progress: 30.0% (1:50:16.930638) Train Loss: 5.93277 Val Loss: 6.96684 Train Perplexity: 3.77e+02 Val Perplexity: 1.06e+03 Learning rate: 0.001 


100%|██████████| 817/817 [20:39<00:00,  1.52s/it]


AutoReconnect: aihub.pozalabs.com:30877: [Errno 54] Connection reset by peer

In [14]:
%debug

> /Users/dreamgonfly/anaconda3/lib/python3.6/site-packages/torch/tensor.py(320)__rdiv__()
    318 
    319     def __rdiv__(self, other):
--> 320         return self.reciprocal() * other
    321     __rtruediv__ = __rdiv__
    322     __itruediv__ = _C._TensorBase.__idiv__



ipdb>  train_batch_losses


*** NameError: name 'train_batch_losses' is not defined


ipdb>  u


> /Users/dreamgonfly/Documents/pozalabs/infinityWord/trainers/seq2seq_trainer.py(70)train()
     68 
     69         train_token_counts = sum(train_batch_token_counts)
---> 70         train_epoch_loss_per_token = sum(train_batch_losses) / train_token_counts
     71 
     72         if epoch % self.print_every == 0:



...

ipdb>  sum(train_batch_losses).dtype


*** AttributeError: 'float' object has no attribute 'dtype'


ipdb>  train_token_counts


tensor(246)


ipdb>  q


In [15]:
[1]*(-1)

[]

In [14]:
%debug

> /Users/dreamgonfly/Documents/pozalabs/infinityWord/trainers/seq2seq_trainer.py(209)seq2seq_collate_fn()
    207 
    208     sources_tensor = torch.tensor(sources_padded)
--> 209     targets_tensor = torch.tensor(targets_padded)
    210 
    211     inputs_tensor = targets_tensor[:, :-1].contiguous()



ipdb>  [len(a) for a in sources_padded]


[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]


ipdb>  [len(a) for a in targets_padded]


[19, 19, 20, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]


...

ipdb>  len(sorted_batch[2][1])


14


ipdb>  longest_target_length


19


ipdb>  q


In [ ]:
import torch

In [ ]:
from torch.nn.functional import softmax

In [ ]:
from dictionaries import BaseDictionary

In [ ]:
dictionary = BaseDictionary.load('base_dictionary')

In [ ]:
inputs = torch.tensor([dictionary['<StartSent>']]).unsqueeze(0)
inputs

In [ ]:
model.eval()

In [39]:
dictionary.idx2word[softmax(model(inputs), dim=2).argmax().item()]

NameError: name 'softmax' is not defined

In [40]:
from generators import OneSentenceGenerator

ImportError: cannot import name 'OneSentenceGenerator'

In [41]:
generator = OneSentenceGenerator(model, dictionary, checkpoint_filepath='parameters/LSTM/LSTM_0_2018-07-28 19:31:26.237748.pth')

NameError: name 'OneSentenceGenerator' is not defined

In [42]:
generator.generate_one()

NameError: name 'generator' is not defined